In [ ]:
from datetime import datetime
from meteostat import Point, Hourly
import pandas as pd
import os

start = datetime(2025, 1, 1)
end = datetime(2025, 1, 31, 23, 59)

airport_coords = {
    'ATL': (33.6407, -84.4277),
    'DFW': (32.8998, -97.0403),
    'DEN': (39.8561, -104.6737),
    'ORD': (41.9742, -87.9073),
    'LAX': (33.9416, -118.4085),
    'JFK': (40.6413, -73.7781),
    'LAS': (36.0840, -115.1537),
    'MCO': (28.4312, -81.3081),
    'MIA': (25.7959, -80.2870),
    'CLT': (35.2144, -80.9473),
    'SEA': (47.4502, -122.3088),
    'SFO': (37.6213, -122.3790)
}

dfs = []
for code, (lat, lon) in airport_coords.items():
    print(f"Downloading weather for {code}...", end=" ")
    location = Point(lat, lon)
    data = Hourly(location, start, end)
    data = data.fetch()
    data = data.reset_index()
    data['Airport'] = code  
    cols_to_keep = ['time', 'Airport', 'temp', 'rhum', 'prcp', 'wspd', 'coco']
    available_cols = [c for c in cols_to_keep if c in data.columns]
    data = data[available_cols]
    dfs.append(data)
    print("OK.")     

if dfs:
    weather_final = pd.concat(dfs)
    output_dir = "../data/raw"
    os.makedirs(output_dir, exist_ok=True)
    output_path = f"{output_dir}/weather_2025_01.csv"
    weather_final.to_csv(output_path, index=False)
    print("-" * 30)
    print(f"Total rows: {len(weather_final)}")
    print(weather_final.head())
else:
    print("No data could be downloaded")